In [1]:
import pandas as pd
import requests

In [2]:
textile_df = pd.read_csv('../data/Fabric Sustainability Data - Textile Research.csv')
textile_df.head()

,material_id,conventional_material_id,source,preferred,biodegradable,microplastics,deforestation,forest_management,chemical_hazard,closed_loop_process,animal_cruelty,recycled,agricultural_impact,organic
0,COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False
1,RECYCLED_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,True,False,False
2,ORGANIC_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,False,False,True
3,EGYPTIAN_COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False
4,PIMA_COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False


In [3]:
material_params = ["COTTON", "RECYCLED_COTTON", "ORGANIC_COTTON", "POLYESTER", "RECYCLED_POLYESTER", "NYLON", "RECYCLED_NYLON", "ACRYLIC", "SPANDEX", "FLAX", "LINEN", "HEMP", "CUPRO", "LYOCELL", "TENCEL_LYOCELL_LENZING", "REFIBRA_TENCEL_LYOCELL_LENZING", "MODAL", "TENCEL_MODAL_LENZING", "VISCOSE", "VISCOSE_BAMBOO", "VISCOSE_ASIA_LENZING", "VISCOSE_EU_LENZING", "SILK", "ALPACA", "WOOL", "RECYCLED_WOOL", "CASHMERE", "RECYCLED_CASHMERE"]

In [4]:
# only keep relevant rows (where material is in dropdown list)
textile_df = textile_df[textile_df['material_id'].isin(material_params)]
textile_df.head(10)

,material_id,conventional_material_id,source,preferred,biodegradable,microplastics,deforestation,forest_management,chemical_hazard,closed_loop_process,animal_cruelty,recycled,agricultural_impact,organic
0,COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False
1,RECYCLED_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,True,False,False
2,ORGANIC_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,False,False,True
6,POLYESTER,POLYESTER,synthetic,False,False,True,False,False,False,False,False,False,False,False
7,RECYCLED_POLYESTER,POLYESTER,synthetic,True,False,True,False,False,False,False,False,True,False,False
8,ACRYLIC,ACRYLIC,synthetic,False,False,True,False,False,False,False,False,False,False,False
9,ALPACA,ALPACA,animal,False,True,False,False,False,False,False,True,False,False,False
10,SPANDEX,SPANDEX,synthetic,False,False,True,False,False,False,False,False,False,False,False
11,FLAX,FLAX,plant,False,True,False,False,False,False,False,False,False,False,False
13,LINEN,LINEN,plant,False,True,False,False,False,False,False,False,False,False,False


In [10]:
# item is in this format: item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

def get_material_score(item) -> float:
    # keep only materials and percentages
    materials_and_percents = {(key.replace(' ', '_')).upper(): value for key, value in item.items() if key != 'item'}

    # weights
    material_weight = 0.5
    end_of_life_weight = 0.5

    # variables for percentage sums
    material_percent_sum = 0
    eol_percent_sum = 0

    # go through each material and its percentage
    for material, percent in materials_and_percents.items():
        
        # convert percentage to decimal
        normalized_percent = percent / 100
        # print(material, normalized_percent)

        # get whether the material is "preferred"
        preferred_value = textile_df.loc[textile_df['material_id'] == material, 'preferred'].values[0]
        # print(f'preferred: {preferred_value}')

        # if it is, add the material percentage to sum
        if preferred_value:
            material_percent_sum += normalized_percent

        # get whether the material is "biodegradable"
        biodegradable_value = textile_df.loc[textile_df['material_id'] == material, 'biodegradable'].values[0]
        # print(f'biodegradable: {biodegradable_value}')

        # if it is, add the material percentage to sum
        if biodegradable_value:
            eol_percent_sum += normalized_percent

    # get material score after multiplying by weight
    material_score = material_weight * material_percent_sum

    # get end of life score after multiplying by weight
    end_of_life_score = end_of_life_weight * eol_percent_sum

    # get final score out of 5
    final_score = 5 * (material_score + end_of_life_score)
    # print(f'final score out of 5: {final_score}')

    return final_score

In [11]:
# example from material_calculator.ipynb
item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

get_material_score(item)

2.5

In [9]:
# getting product rank (out of 1)
url = 'https://api.selflessclothes.com/research/calculator?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14'

response = requests.get(url)
data = response.json()

print(data.get('rank')) # 0.08

# https://www.selflessclothes.com/blog/sustainability-calculator/?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14
# note: for these materials, the score out of 5 is 0.4 (0.08 * 5)

0.08
